In [28]:
import os 
import numpy as np
import pandas as pd

all_session =[i for i in range(1,31)]
print(all_session)
cv_dataset = {}
for i in range(0,10): # CV = 10
    if i <7:
        cv_dataset[f"CV_{i+1}"] = {
            "Test":all_session[i*3:3*(i+1)],
            "Validation":all_session[(i+1)*3:(i+1)*3+9],
            "Train":list(set(all_session) - set(all_session[i*3:3*(i+1)])- set(all_session[(i+1)*3:(i+1)*3+6]))
        }
    if i ==7:
        cv_dataset[f"CV_{i+1}"] = {
            "Test":all_session[i*3:3*(i+1)],
            "Validation":[25,26,27,28,29,30,1,2,3],
            "Train":list(set(all_session) - set(all_session[i*3:3*(i+1)])- set([25,26,27,28,29,30,1,2,3]))
        }
    if i ==8:
        cv_dataset[f"CV_{i+1}"] = {
            "Test":all_session[i*3:3*(i+1)],
            "Validation":[28,29,30,1,2,3,4,5,6],
            "Train":list(set(all_session) - set(all_session[i*3:3*(i+1)])- set([28,29,30,1,2,3,4,5,6]))
        }
    if i ==9:
        cv_dataset[f"CV_{i+1}"] = {
            "Test":all_session[i*3:3*(i+1)],
            "Validation":[1,2,3,4,5,6,7,8,9],
            "Train":list(set(all_session) - set(all_session[i*3:3*(i+1)])- set([1,2,3,4,5,6,7,8,9]))
        }
        

for idx,item in enumerate(cv_dataset):
    print(f"CV = {idx+1}")
    print(f"Test >>> {cv_dataset[item]['Test']} Validation >>> {cv_dataset[item]['Validation']}")
    print(f"Train >>> {cv_dataset[item]['Train']}\n")

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
CV = 1
Test >>> [1, 2, 3] Validation >>> [4, 5, 6, 7, 8, 9, 10, 11, 12]
Train >>> [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

CV = 2
Test >>> [4, 5, 6] Validation >>> [7, 8, 9, 10, 11, 12, 13, 14, 15]
Train >>> [1, 2, 3, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

CV = 3
Test >>> [7, 8, 9] Validation >>> [10, 11, 12, 13, 14, 15, 16, 17, 18]
Train >>> [1, 2, 3, 4, 5, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

CV = 4
Test >>> [10, 11, 12] Validation >>> [13, 14, 15, 16, 17, 18, 19, 20, 21]
Train >>> [1, 2, 3, 4, 5, 6, 7, 8, 9, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

CV = 5
Test >>> [13, 14, 15] Validation >>> [16, 17, 18, 19, 20, 21, 22, 23, 24]
Train >>> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 22, 23, 24, 25, 26, 27, 28, 29, 30]

CV = 6
Test >>> [16, 17, 18] Validation >>> [19,

In [29]:
len(cv_dataset)

10

# Add Session Infomation

In [14]:
# Add Session Information, 先自己加一行 Session列
features = pd.read_csv('./1_Features_raw/Regression_Speaker_Visual_hog_final_session.csv')
for idx, item in features.iterrows():
    session = int(item['VideoName'][:2])
    features.iloc[idx, 0] = session
features.to_csv('./1_Features_raw/Regression_Speaker_Visual_hog_final_session2.csv')

# Create CV dataset 
## (1) For Classification

In [30]:
# manually delete the first two cols
features = pd.read_csv('./1_Features_raw/Classification_Speaker_Visual_Hog_6PNN_session2.csv')
participate = pd.read_csv("Final_results_0620_1721.csv")
for cv_idx, key in enumerate(cv_dataset):
    Test_sessions = cv_dataset[key]["Test"]
    print("Test sessions = ",Test_sessions )
    Validation_sessions = cv_dataset[key]["Validation"]
    Train_sessions = cv_dataset[key]["Train"]
    
    # ---------------------
    # load data
    # ---------------------
    Test_data = features[features['Session'].isin(Test_sessions)]
    Test_data.to_csv(f"./CV_Features_631/ClassificationFeatures/Test_CV_{cv_idx+1}.csv")
    Validation_data = features[features['Session'].isin(Validation_sessions)]
    Train_data = features[features['Session'].isin(Train_sessions)]

    # ---------------------
    # speaker independent
    # ---------------------
    # (1) delete instance
    delete_info = set()
    Test_Overlap = participate[participate['Round'].isin(cv_dataset[key]["Test"])]
    for idx, item in Test_Overlap.iterrows():
        values = set(item.values[6:-1])
        for temp in values:
            if type(temp)!=float:
                if int(temp.split('-')[0]) not in Test_sessions:
                    if int(temp.split('-')[0])<10:
                        temp = '0'+ temp
                        delete_info.add(temp.replace("-","_"))
                    else:
                         delete_info.add(temp.replace("-","_"))
    print(delete_info)
    
    # (2) 整理Validation and Train data
    for idx, item in Validation_data.iterrows():
        if item['VideoName'][:5] in list(delete_info):
            Validation_data.drop(labels = idx,inplace=True)
    Validation_data.to_csv(f"./CV_Features_631/ClassificationFeatures/Validation_CV_{cv_idx+1}.csv")
    
    for idx, item in Train_data.iterrows():
        if item['VideoName'][:5] in list(delete_info):
            Train_data.drop(labels = idx,inplace=True)
    Train_data.to_csv(f"./CV_Features_631/ClassificationFeatures/Train_CV_{cv_idx+1}.csv")

Test sessions =  [1, 2, 3]
{'20_03', '12_02', '20_02', '06_08', '09_09', '04_01', '22_09', '05_08', '04_02', '13_01', '14_09', '21_04', '09_01', '09_07', '16_02', '20_06', '17_06', '20_05', '05_04', '09_06', '07_03', '18_02', '21_07', '08_05', '15_02', '16_04', '21_02', '11_03', '08_04', '04_04', '10_05', '08_03', '13_08', '07_05', '18_01', '21_08', '17_04', '16_08', '12_05'}


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Test sessions =  [4, 5, 6]
{'11_01', '14_07', '12_04', '11_09', '21_04', '11_02', '09_07', '14_06', '10_07', '23_01', '10_09', '12_01', '22_06', '07_04', '10_04', '18_03', '03_06', '11_07', '10_02', '21_01', '20_01', '24_04', '15_07', '15_02', '15_05', '02_09', '12_07', '18_04', '13_09', '11_04', '10_05', '08_03', '03_09', '07_05', '03_03', '11_08', '09_03', '16_09', '17_04', '28_01', '08_08', '01_01', '16_08'}
Test sessions =  [7, 8, 9]
{'12_02', '20_02', '11_06', '02_05', '04_05', '18_08', '22_05', '14_09', '13_07', '13_01', '16_02', '06_09', '14_02', '17_06', '20_06', '18_06', '05_04', '20_05', '24_09', '11_05', '10_04', '18_03', '02_08', '10_08', '03_06', '21_07', '21_01', '01_09', '20_01', '10_03', '16_04', '12_03', '21_09', '17_08', '03_05', '13_06', '04_04', '03_01', '12_08', '12_09', '20_09', '11_08', '18_07', '18_01', '01_02', '01_01', '05_05', '12_05'}
Test sessions =  [10, 11, 12]
{'08_09', '04_01', '14_07', '22_09', '07_02', '13_07', '21_04', '18_08', '17_03', '08_01', '14_

## (2) For Regression

In [31]:
# manually delete the first two cols
features = pd.read_csv('./1_Features_raw/Regression_Speaker_Visual_hog_final_session2.csv')
participate = pd.read_csv("Final_results_0620_1721.csv")
for cv_idx, key in enumerate(cv_dataset):
    Test_sessions = cv_dataset[key]["Test"]
    print("Test sessions = ",Test_sessions )
    Validation_sessions = cv_dataset[key]["Validation"]
    Train_sessions = cv_dataset[key]["Train"]
    
    # ---------------------
    # load data
    # ---------------------
    Test_data = features[features['Session'].isin(Test_sessions)]
    Test_data.to_csv(f"./CV_Features_631/RegressionFeatures/Test_CV_{cv_idx+1}.csv")
    Validation_data = features[features['Session'].isin(Validation_sessions)]
    Train_data = features[features['Session'].isin(Train_sessions)]

    # ---------------------
    # speaker independent
    # ---------------------
    # (1) delete instance
    delete_info = set()
    Test_Overlap = participate[participate['Round'].isin(cv_dataset[key]["Test"])]
    for idx, item in Test_Overlap.iterrows():
        values = set(item.values[6:-1])
        for temp in values:
            if type(temp)!=float:
                if int(temp.split('-')[0]) not in Test_sessions:
                    if int(temp.split('-')[0])<10:
                        temp = '0'+ temp
                        delete_info.add(temp.replace("-","_"))
                    else:
                         delete_info.add(temp.replace("-","_"))
    print(delete_info)
    
    # (2) 整理Validation and Train data
    for idx, item in Validation_data.iterrows():
        if item['VideoName'][:5] in list(delete_info):
            Validation_data.drop(labels = idx,inplace=True)
    Validation_data.to_csv(f"./CV_Features_631/RegressionFeatures/Validation_CV_{cv_idx+1}.csv")
    
    for idx, item in Train_data.iterrows():
        if item['VideoName'][:5] in list(delete_info):
            Train_data.drop(labels = idx,inplace=True)
    Train_data.to_csv(f"./CV_Features_631/RegressionFeatures/Train_CV_{cv_idx+1}.csv")

Test sessions =  [1, 2, 3]
{'20_03', '12_02', '20_02', '06_08', '09_09', '04_01', '22_09', '05_08', '04_02', '13_01', '14_09', '21_04', '09_01', '09_07', '16_02', '20_06', '17_06', '20_05', '05_04', '09_06', '07_03', '18_02', '21_07', '08_05', '15_02', '16_04', '21_02', '11_03', '08_04', '04_04', '10_05', '08_03', '13_08', '07_05', '18_01', '21_08', '17_04', '16_08', '12_05'}


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Test sessions =  [4, 5, 6]
{'11_01', '14_07', '12_04', '11_09', '21_04', '11_02', '09_07', '14_06', '10_07', '23_01', '10_09', '12_01', '22_06', '07_04', '10_04', '18_03', '03_06', '11_07', '10_02', '21_01', '20_01', '24_04', '15_07', '15_02', '15_05', '02_09', '12_07', '18_04', '13_09', '11_04', '10_05', '08_03', '03_09', '07_05', '03_03', '11_08', '09_03', '16_09', '17_04', '28_01', '08_08', '01_01', '16_08'}
Test sessions =  [7, 8, 9]
{'12_02', '20_02', '11_06', '02_05', '04_05', '18_08', '22_05', '14_09', '13_07', '13_01', '16_02', '06_09', '14_02', '17_06', '20_06', '18_06', '05_04', '20_05', '24_09', '11_05', '10_04', '18_03', '02_08', '10_08', '03_06', '21_07', '21_01', '01_09', '20_01', '10_03', '16_04', '12_03', '21_09', '17_08', '03_05', '13_06', '04_04', '03_01', '12_08', '12_09', '20_09', '11_08', '18_07', '18_01', '01_02', '01_01', '05_05', '12_05'}
Test sessions =  [10, 11, 12]
{'08_09', '04_01', '14_07', '22_09', '07_02', '13_07', '21_04', '18_08', '17_03', '08_01', '14_